In [8]:
import time
import numpy as np
import tkinter as tk

import cv2
import mediapipe as mp
from PIL import Image, ImageTk

from tkinter_webcam import webcam

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(3,640) # adjust width
cap.set(4,480) # adjust height


mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=4)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

pTime = 0
cTime = 0

while True:
    success, img = cap.read()
    img = cv2.flip(img,1)
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    # print(results.multi_hand_landmarks)
    if results.multi_hand_worlandmarks: 
        for handLMS in results.multi_hand_landmarks:
            for id, lm in enumerate(handLMS.landmark):
                h,w,c = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                if id == 4:
                    cv2.circle(img,(cx,cy),15,(255,0,255),cv2.FILLED)
                
            mp_drawing.draw_landmarks(img,handLMS,mpHands.HAND_CONNECTIONS)
    
    #Get FPS
    cTime = time.time()
    fps = 1/(cTime - pTime)
    pTime = cTime
    
    cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_COMPLEX,3,(255,0,255),3)
    
    cv2.imshow("Webcam", img) # This will open an independent window
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cap.release()
        break
cv2.destroyAllWindows() 

In [9]:
class handDetector():
    def __init__(self,mode=False, maxHands = 2, detectionCon=0.5,trackCon=0.5,model_complexity=0):
        #initialize the configuration for mp Hands
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        self.model_complexity = model_complexity
        self.mpHands = mp.solutions.hands
        #Construct the hand detection model
        self.hands = self.mpHands.Hands(static_image_mode = self.mode,
                                        max_num_hands = self.maxHands,
                                        min_detection_confidence = self.detectionCon,
                                        min_tracking_confidence = self.trackCon,
                                        model_complexity = self.model_complexity)
        
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        
    def _get_output_path(self, name):
            return os.path.join(tempfile.gettempdir(), self.id().split('.')[-1] + name)

    def _landmarks_list_to_array(self, landmark_list, image_shape):
        rows, cols, _ = image_shape
        return np.asarray([(lmk.x * cols, lmk.y * rows, lmk.z * cols)
                        for lmk in landmark_list.landmark])

    def _world_landmarks_list_to_array(self, landmark_list):
        return np.asarray([(lmk.x, lmk.y, lmk.z)
                        for lmk in landmark_list.landmark])


    def findHands(self, img,draw = True):
        #simple method for finding and displaying a hand on an image
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)
        if self.results.multi_hand_landmarks: 
            for handLMS in self.results.multi_hand_landmarks:
                if draw:
                    self.mp_drawing.draw_landmarks(img,
                                                   handLMS,
                                                   self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self, img, handID=0, draw = True):
        #list will have all the landmark positions
        lmList = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handID] #[self.results.multi_hand_landmarks[id] for id in handID]
            
            for id, lm in enumerate(myHand.landmark):
                h,w,c = img.shape
                cx,cy = int(lm.x*w),int(lm.y*h)
                lmList.append([id,cx,cy])
                # if id == 4:
                if draw:
                    cv2.circle(img,(cx,cy),15,(255,0,255),cv2.FILLED)
                
            
        return(lmList)
    
    def findRealWorldPositions(self, img, handID=0):
        lmList = []
        if self.results.multi_hand_world_landmarks:
            myHand = self.results.multi_hand_world_landmarks[handID]
            for id,hand_world_landmarks in enumerate(myHand.landmark):
                cx,cy = int(hand_world_landmarks.x),int(hand_world_landmarks.y)
                lmList.append([id,cx,cy])
        return(lmList)

In [10]:
#this would be in main
pTime = 0
cTime = 0
cap = cv2.VideoCapture(0)
cap.set(3,1280) # adjust width
cap.set(4,960) # adjust height|

detector = handDetector()
while True:
    success, img = cap.read()
    img = cv2.flip(img,1)
    img.flags.writeable = False
    img = detector.findHands(img)
    lmlist = detector.findPosition(img)
    rw_list = detector.findRealWorldPositions(img)
    #Get FPS
    cTime = time.time()
    fps = 1/(cTime - pTime)
    pTime = cTime
    
    cv2.putText(img,str(int(fps)),(10,70),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,255),1)
    if len(lmlist) != 0:
        cv2.putText(img,f'Thumb: ({str(int(lmlist[4][1]))},{str(int(lmlist[4][2]))})',(10,100),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),1)
        cv2.putText(img,f'Index: ({str(int(lmlist[8][1]))},{str(int(lmlist[8][2]))})',(10,130),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),1)
        
    
    cv2.imshow("Webcam", img) # This will open an independent window
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cap.release()
        break
cv2.destroyAllWindows() 

In [7]:
rw_list

[]

In [5]:
# This is using some simple package that handles image to photoimage conversion etc
# window = tk.Tk()
# window.title('Webcam Live Feed')
# window.geometry('1300x500')
# video = webcam.Box(window,width=450,height=450)
# video.show_frames()

# tk.mainloop()

# Lets start building the App for the Demo

Design goals: 
- Have Video stream from webcam
- Have buttons to select which test (FTA or WRT)
- Have buttons to start and stop data recording
- will display live update of tests for each participant
- Upon stop will populate or update the Table of average metrics

Will try using Tkinter to design the application


In [3]:
#Step 1 get the video capture object from the camera
cap = cv2.VideoCapture(0)